## Multi-Agent Customer Outreach Campaign

In [25]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [26]:
from crewai import Crew, Task, Agent

In [27]:
import os
from utils import get_openai_api_key
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [28]:
#Creating Agents
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
        "our ideal customer profile", #ICP
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [29]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [30]:
## Crew_Ai Tools
from crewai_tools import DirectoryReadTool, \
                        FileReadTool, \
                        SerperDevTool

directory_read_tool = DirectoryReadTool(directory='./instructions') #Read the directory
file_read_tool = FileReadTool() #Read the file
search_tool = SerperDevTool() # Search Internet

In [31]:
## Custom Tool
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
        "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

sentiment_analysis_tool = SentimentAnalysisTool()

In [32]:
## Tasks

lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [33]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [34]:
# Crew
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
        personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2025-03-25 18:04:39,268 - 8581613632 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [37]:
inputs = {
    "lead_name": "NusterAI",
    "industry": " Business with Cognitive Intelligence",
    "key_decision_maker": "ABC",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of NusterAI, a company in the  Business with Cognitive Intelligence sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To conduct an in-depth analysis of NusterAI, I need to gather detailed information about the company, including its background, key personnel, recent milestones, and identified needs. Since I need to use available data sources, I will start by searching the internet to gather this information about NusterAI.

Action: Search the internet
Action Input: {"search_query": "NusterAI company background ke

In [38]:
from IPython.display import Markdown
Markdown(result)

**Email Draft 1: Introduction and Value Proposition**

Subject: Empowering NusterAI's Vision with CrewAI's Advanced Solutions

Hi ABC,

I hope this message finds you well. Congratulations on the recent product launch at NusterAI! It's truly inspiring to see how your team is revolutionizing AI collaboration and empowering businesses with advanced autonomous AI agents.

At CrewAI, we share a similar passion for innovation in the AI space. Our AI-driven solutions are designed to enhance efficiency and drive growth, tailored to meet the unique needs of organizations like NusterAI. We offer a suite of tools, including advanced data analytics and personalized learning experiences, that could complement and elevate the capabilities of your new product.

I would love the opportunity to discuss how CrewAI can support NusterAI's goals and contribute to your continued success. Are you available for a brief call next week?

Looking forward to the possibility of collaborating.

Warm regards,

[Your Name]  
Lead Sales Representative  
CrewAI  

**Email Draft 2: Highlighting Potential Collaboration**

Subject: Unlock New Potentials with CrewAI's Innovative Solutions

Hello ABC,

Congratulations on NusterAI's exciting new venture! Launching a product that promises to transform AI collaboration is no small feat, and your team's dedication is commendable.

At CrewAI, we specialize in creating AI-driven solutions that not only enhance current capabilities but also open doors to new possibilities. Our customer relationship management and data analytics tools are designed to seamlessly integrate with cutting-edge technologies, providing actionable insights and fostering stronger client relationships.

I believe there could be great synergy between our offerings and NusterAI's innovative drive. Could we schedule a time to explore how CrewAI's solutions might align with your vision?

Thank you for considering this partnership.

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI  

**Email Draft 3: Focus on Shared Vision and Future Goals**

Subject: Shared Vision for Future Innovation

Dear ABC,

I hope you're doing well. The recent launch at NusterAI marks a significant milestone in advancing AI technology, and I wanted to extend my congratulations to you and your team.

At CrewAI, we are driven by a shared vision of leveraging AI to create smarter, more efficient workflows. Our tools are designed to adapt and evolve, much like the innovative solutions at NusterAI. By integrating our AI-driven platforms, we can support your mission to lead in AI collaboration and achieve even greater milestones.

I'd be thrilled to discuss how CrewAI can contribute to NusterAI's future goals. Could we set up a time to connect in the coming days?

Thank you, and I look forward to the opportunity.

Sincerely,

[Your Name]  
Lead Sales Representative  
CrewAI  

These drafts aim to initiate a conversation with ABC by highlighting the value and synergy between NusterAI's innovations and CrewAI's offerings.